In [9]:
!pip install pygame

In [ ]:
import pygame, sys, random
from pygame.locals import *

# Create the constants (go ahead and experiment with different values)
BOARDWIDTH = 5  # number of columns in the board
BOARDHEIGHT = 5 # number of rows in the board
TILESIZE = 60
WINDOWWIDTH = 1500
WINDOWHEIGHT = 500
FPS = 30
BLANK = None

#                 R    G    B
BLACK =         (  0,   0,   0)
WHITE =         (255, 255, 255)
BRIGHTBLUE =    (  0,  50, 255)
DARKTURQUOISE = (  3,  54,  73)
GREEN =         (  0, 204,   0)

BGCOLOR = DARKTURQUOISE
TILECOLOR = GREEN
TEXTCOLOR = WHITE
BORDERCOLOR = BRIGHTBLUE
BASICFONTSIZE = 20

BUTTONCOLOR = WHITE
BUTTONTEXTCOLOR = BLACK
MESSAGECOLOR = WHITE

XMARGIN = int((WINDOWWIDTH - (TILESIZE * BOARDWIDTH + (BOARDWIDTH - 1))) / 2)
YMARGIN = int((WINDOWHEIGHT - (TILESIZE * BOARDHEIGHT + (BOARDHEIGHT - 1))) / 2)

UP = 'up'
DOWN = 'down'
LEFT = 'left'
RIGHT = 'right'

# Define table types
REGULAR = 'regular'
MULTIPLICATION = 'multiplication'
ADDITION = 'addition'  
FIBONACCI = 'fibonacci'
BACKWARD_REGULAR = 'backward_regular'

def main():
    global FPSCLOCK, DISPLAYSURF, BASICFONT,SOLVE_SURF, SOLVE_RECT
    global REGULAR_SURF, REGULAR_RECT, MULTIPLICATION_SURF, MULTIPLICATION_RECT
    global ADDITION_SURF, ADDITION_RECT,FIBONACCI_SURF,FIBONACCI_RECT, BACKWARD_REGULAR_SURF, BACKWARD_REGULAR_RECT

    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    pygame.display.set_caption('Slide Puzzle')
    BASICFONT = pygame.font.Font('freesansbold.ttf', BASICFONTSIZE)

    # Store the option buttons and their rectangles in OPTIONS.
    SOLVE_SURF, SOLVE_RECT = makeText('Solve',    TEXTCOLOR, TILECOLOR, WINDOWWIDTH - 150, WINDOWHEIGHT - 90)

    REGULAR_SURF, REGULAR_RECT = makeText('Regular', TEXTCOLOR, TILECOLOR, WINDOWWIDTH - 150, WINDOWHEIGHT - 240)
    MULTIPLICATION_SURF, MULTIPLICATION_RECT = makeText('Multiplication', TEXTCOLOR, TILECOLOR, WINDOWWIDTH - 150, WINDOWHEIGHT - 210)
    ADDITION_SURF, ADDITION_RECT = makeText('Addition', TEXTCOLOR, TILECOLOR, WINDOWWIDTH - 150, WINDOWHEIGHT - 180)  
    FIBONACCI_SURF, FIBONACCI_RECT = makeText('Fibonacci', TEXTCOLOR, TILECOLOR, WINDOWWIDTH - 150, WINDOWHEIGHT - 120)  
    BACKWARD_REGULAR_SURF, BACKWARD_REGULAR_RECT = makeText('Backwards', TEXTCOLOR, TILECOLOR, WINDOWWIDTH - 150, WINDOWHEIGHT - 150)  

    mainBoard, solutionSeq = generateNewPuzzle(80, REGULAR)  # Start with regular puzzle
    SOLVEDBOARD = getStartingBoard(BOARDWIDTH, BOARDHEIGHT)  # a solved board is the same as the board in a start state.
    allMoves = []  # list of moves made from the solved configuration

    while True:  # main game loop
        slideTo = None  # the direction, if any, a tile should slide
        msg = 'Click tile or press arrow keys to slide.'  # contains the message to show in the upper left corner.
        if mainBoard == SOLVEDBOARD:
            msg = 'Solved!'

        drawBoard(mainBoard, msg)

        checkForQuit()
        for event in pygame.event.get():  # event handling loop
            if event.type == MOUSEBUTTONUP:
                spotx, spoty = getSpotClicked(mainBoard, event.pos[0], event.pos[1])

                if (spotx, spoty) == (None, None):
                    # check if the user clicked on an option button
                    if SOLVE_RECT.collidepoint(event.pos):
                        resetAnimation(mainBoard, solutionSeq + allMoves)  # clicked on Solve button
                        allMoves = []
                    elif REGULAR_RECT.collidepoint(event.pos):
                        mainBoard, solutionSeq = generateNewPuzzle(80, REGULAR)
                        allMoves = []
                    elif MULTIPLICATION_RECT.collidepoint(event.pos):
                        mainBoard, solutionSeq = generateNewPuzzle(80, MULTIPLICATION)
                        allMoves = []
                    elif ADDITION_RECT.collidepoint(event.pos):  # Handle event for Addition button
                        mainBoard, solutionSeq = generateNewPuzzle(80, ADDITION)
                        allMoves = []
                    elif FIBONACCI_RECT.collidepoint(event.pos):  # Handle event for Fibonacci button
                        mainBoard, solutionSeq = generateNewPuzzle(80, FIBONACCI)
                        allMoves = []
                    elif BACKWARD_REGULAR_RECT.collidepoint(event.pos):  # Handle event for backward regular button
                        mainBoard, solutionSeq = generateNewPuzzle(80, BACKWARD_REGULAR)
                        allMoves = []
                else:
                    # check if the clicked tile was next to the blank spot

                    blankx, blanky = getBlankPosition(mainBoard)
                    if spotx == blankx + 1 and spoty == blanky:
                        slideTo = LEFT
                    elif spotx == blankx - 1 and spoty == blanky:
                        slideTo = RIGHT
                    elif spotx == blankx and spoty == blanky + 1:
                        slideTo = UP
                    elif spotx == blankx and spoty == blanky - 1:
                        slideTo = DOWN

            elif event.type == KEYUP:
                # check if the user pressed a key to slide a tile
                if event.key in (K_LEFT, K_a) and isValidMove(mainBoard, LEFT):
                    slideTo = LEFT
                elif event.key in (K_RIGHT, K_d) and isValidMove(mainBoard, RIGHT):
                    slideTo = RIGHT
                elif event.key in (K_UP, K_w) and isValidMove(mainBoard, UP):
                    slideTo = UP
                elif event.key in (K_DOWN, K_s) and isValidMove(mainBoard, DOWN):
                    slideTo = DOWN

        if slideTo:
            slideAnimation(mainBoard, slideTo, 'Click tile or press arrow keys to slide.', 8)  # show slide on screen
            makeMove(mainBoard, slideTo)
            allMoves.append(slideTo)  # record the slide
        pygame.display.update()
        FPSCLOCK.tick(FPS)


def terminate():
    pygame.quit()
    sys.exit()


def checkForQuit():
    for event in pygame.event.get(QUIT):  # get all the QUIT events
        terminate()  # terminate if any QUIT events are present
    for event in pygame.event.get(KEYUP):  # get all the KEYUP events
        if event.key == K_ESCAPE:
            terminate()  # terminate if the KEYUP event was for the Esc key
        pygame.event.post(event)  # put the other KEYUP event objects back


def getStartingBoard(width, height):
    # Return a board data structure with tiles in the solved state.
    # For example, if BOARDWIDTH and BOARDHEIGHT are both 3, this function
    # returns [[1, 4, 7], [2, 5, 8], [3, 6, BLANK]]
    counter = 1
    board = []
    for x in range(width):
        column = []
        for y in range(height):
            column.append(counter)
            counter += width
        board.append(column)
        counter -= width * (height - 1) + width - 1

    board[width-1][height-1] = BLANK
    return board


def getBlankPosition(board):
    # Return the x and y of board coordinates of the blank space.
    for x in range(len(board)):
        for y in range(len(board[0])):
            if board[x][y] == BLANK:
                return (x, y)


def makeMove(board, move):
    # This function does not check if the move is valid.
    blankx, blanky = getBlankPosition(board)

    if move == UP:
        board[blankx][blanky], board[blankx][blanky + 1] = board[blankx][blanky + 1], board[blankx][blanky]
    elif move == DOWN:
        board[blankx][blanky], board[blankx][blanky - 1] = board[blankx][blanky - 1], board[blankx][blanky]
    elif move == LEFT:
        board[blankx][blanky], board[blankx + 1][blanky] = board[blankx + 1][blanky], board[blankx][blanky]
    elif move == RIGHT:
        board[blankx][blanky], board[blankx - 1][blanky] = board[blankx - 1][blanky], board[blankx][blanky]
        
def isValidMove(board, move):
    # Function to check if a move is valid on the board
    blankx, blanky = getBlankPosition(board)  # Get the position of the blank tile
    # Check if the move is valid based on the direction and position of the blank tile
    return (move == UP and blanky != len(board[0]) - 1) or \
           (move == DOWN and blanky != 0) or \
           (move == LEFT and blankx != len(board) - 1) or \
           (move == RIGHT and blankx != 0)


def getRandomMove(board, lastMove=None):
    # start with a full list of all four moves
    validMoves = [UP, DOWN, LEFT, RIGHT]

    # remove moves from the list as they are disqualified
    if lastMove == UP or not isValidMove(board, DOWN):
        validMoves.remove(DOWN)
    if lastMove == DOWN or not isValidMove(board, UP):
        validMoves.remove(UP)
    if lastMove == LEFT or not isValidMove(board, RIGHT):
        validMoves.remove(RIGHT)
    if lastMove == RIGHT or not isValidMove(board, LEFT):
        validMoves.remove(LEFT)

    # return a random move from the list of remaining moves
    return random.choice(validMoves)


def getLeftTopOfTile(tileX, tileY):
    left = XMARGIN + (tileX * TILESIZE) + (tileX - 1)
    top = YMARGIN + (tileY * TILESIZE) + (tileY - 1)
    return (left, top)


def getSpotClicked(board, x, y):
    # Function to get the board coordinates from pixel coordinates (x, y)
    for tileX in range(len(board)):
        for tileY in range(len(board[0])):
            left, top = getLeftTopOfTile(tileX, tileY)  # Get top-left coordinates of the current tile
            tileRect = pygame.Rect(left, top, TILESIZE, TILESIZE)  # Create rectangle for the current tile
            if tileRect.collidepoint(x, y):  # Check if the click point is within the tile rectangle
                return (tileX, tileY)  # Return board coordinates of the clicked tile
    return (None, None)  # Return None if no tile was clicked

def drawTile(tilex, tiley, number, adjx=0, adjy=0):
    # draw a tile at board coordinates tilex and tiley, optionally a few
    # pixels over (determined by adjx and adjy)
    left, top = getLeftTopOfTile(tilex, tiley)
    pygame.draw.rect(DISPLAYSURF, TILECOLOR, (left + adjx, top + adjy, TILESIZE, TILESIZE))
    textSurf = BASICFONT.render(str(number), True, TEXTCOLOR)
    textRect = textSurf.get_rect()
    textRect.center = left + int(TILESIZE / 2) + adjx, top + int(TILESIZE / 2) + adjy
    DISPLAYSURF.blit(textSurf, textRect)


def makeText(text, color, bgcolor, top, left):
    # create the Surface and Rect objects for some text.
    textSurf = BASICFONT.render(text, True, color, bgcolor)
    textRect = textSurf.get_rect()
    textRect.topleft = (top, left)
    return (textSurf, textRect)

def drawBoard(board, message):
    # Function to draw the game board
    DISPLAYSURF.fill(BGCOLOR)  # Fill the display surface with the background color
    if message:
        textSurf, textRect = makeText(message, MESSAGECOLOR, BGCOLOR, 5, 5)  # Create text surface and rectangle for message
        DISPLAYSURF.blit(textSurf, textRect)  # Blit the message onto the display surface

    for tilex in range(len(board)):
        for tiley in range(len(board[0])):
            if board[tilex][tiley]:
                drawTile(tilex, tiley, board[tilex][tiley])  # Draw non-empty tiles

    left, top = getLeftTopOfTile(0, 0)  # Get coordinates for the top-left corner of the board
    width = BOARDWIDTH * TILESIZE  # Calculate width of the board
    height = BOARDHEIGHT * TILESIZE  # Calculate height of the board
    pygame.draw.rect(DISPLAYSURF, BORDERCOLOR, (left - 5, top - 5, width + 11, height + 11), 4)  # Draw board border

    # Display buttons for solving and selecting table types
    DISPLAYSURF.blit(SOLVE_SURF, SOLVE_RECT)
    DISPLAYSURF.blit(REGULAR_SURF, REGULAR_RECT)
    DISPLAYSURF.blit(MULTIPLICATION_SURF, MULTIPLICATION_RECT)
    DISPLAYSURF.blit(ADDITION_SURF, ADDITION_RECT)
    DISPLAYSURF.blit(FIBONACCI_SURF, FIBONACCI_RECT)
    DISPLAYSURF.blit(BACKWARD_REGULAR_SURF, BACKWARD_REGULAR_RECT)  # Display the backward regular table button


def slideAnimation(board, direction, message, animationSpeed):
    # Note: This function does not check if the move is valid.

    blankx, blanky = getBlankPosition(board)
    if direction == UP:
        movex = blankx
        movey = blanky + 1
    elif direction == DOWN:
        movex = blankx
        movey = blanky - 1
    elif direction == LEFT:
        movex = blankx + 1
        movey = blanky
    elif direction == RIGHT:
        movex = blankx - 1
        movey = blanky

    # prepare the base surface
    drawBoard(board, message)
    baseSurf = DISPLAYSURF.copy()
    # draw a blank space over the moving tile on the baseSurf Surface.
    moveLeft, moveTop = getLeftTopOfTile(movex, movey)
    pygame.draw.rect(baseSurf, BGCOLOR, (moveLeft, moveTop, TILESIZE, TILESIZE))

    for i in range(0, TILESIZE, animationSpeed):
        # animate the tile sliding over
        checkForQuit()
        DISPLAYSURF.blit(baseSurf, (0, 0))
        if direction == UP:
            drawTile(movex, movey, board[movex][movey], 0, -i)
        if direction == DOWN:
            drawTile(movex, movey, board[movex][movey], 0, i)
        if direction == LEFT:
            drawTile(movex, movey, board[movex][movey], -i, 0)
        if direction == RIGHT:
            drawTile(movex, movey, board[movex][movey], i, 0)

        pygame.display.update()
        FPSCLOCK.tick(FPS)


def generateNewPuzzle(numSlides, table_type):
    # Function to generate a new puzzle with a specified number of moves and table type
    sequence = []  # Initialize a list to store the sequence of moves
    if table_type == REGULAR:
        board = getStartingBoard(BOARDWIDTH, BOARDHEIGHT)  # Get starting board for regular table
    elif table_type == MULTIPLICATION:
        board = generateMultiplicationTable(BOARDWIDTH, BOARDHEIGHT)  # Generate multiplication table
    elif table_type == ADDITION:
        board = generateAdditionTable(BOARDWIDTH, BOARDHEIGHT)  # Generate addition table
    elif table_type == BACKWARD_REGULAR:
        board = generateBackwardRegularTable(BOARDWIDTH, BOARDHEIGHT)  # Generate backward regular table
    elif table_type == FIBONACCI:
        board = generateFibonacciTable(BOARDWIDTH, BOARDHEIGHT)  # Generate Fibonacci sequence table
    else:
        raise ValueError("Invalid table type")  # Raise error for invalid table type

    drawBoard(board, '')  # Draw the initial board
    pygame.display.update()  # Update display
    pygame.time.wait(500)  # Pause for 500 milliseconds for effect
    lastMove = None  # Initialize last move variable
    for i in range(numSlides):
        move = getRandomMove(board, lastMove)  # Get a random move
        slideAnimation(board, move, 'Generating new puzzle...', animationSpeed=int(TILESIZE / 3))  # Animate the move
        makeMove(board, move)  # Make the move on the board
        sequence.append(move)  # Append the move to the sequence
        lastMove = move  # Update last move
    return (board, sequence)  # Return the final board and sequence of moves

def resetAnimation(board, allMoves):
    # make all of the moves in allMoves in reverse.
    revAllMoves = allMoves[:]  # gets a copy of the list
    revAllMoves.reverse()

    for move in revAllMoves:
        if move == UP:
            oppositeMove = DOWN
        elif move == DOWN:
            oppositeMove = UP
        elif move == RIGHT:
            oppositeMove = LEFT
        elif move == LEFT:
            oppositeMove = RIGHT
        slideAnimation(board, oppositeMove, '', animationSpeed=int(TILESIZE / 2))
        makeMove(board, oppositeMove)


def generateMultiplicationTable(width, height):
    # Function to generate a multiplication table
    board = []  # Initialize an empty board
    for x in range(width):
        column = []  # Initialize an empty column for each row
        for y in range(height):
            # Calculate each cell value by multiplying x and y
            column.append((x + 1) * (y + 1))
        board.append(column)  # Append the column to the board
    board[width - 1][height - 1] = BLANK  # Replace the last cell with BLANK
    return board

def generateAdditionTable(width, height):  
    # Function to generate an addition table
    board = []  # Initialize an empty board
    for x in range(width):
        column = []  # Initialize an empty column for each row
        for y in range(height):
            # Calculate each cell value by adding x and y
            column.append((x + 1) + (y + 1))  # Modify this line for addition operation
        board.append(column)  # Append the column to the board
    board[width - 1][height - 1] = BLANK  # Replace the last cell with BLANK
    return board

def generateFibonacciTable(width, height):
    # Function to generate a Fibonacci sequence table
    board = []  # Initialize an empty board
    a, b = 1, 1  # Initialize Fibonacci sequence variables
    for _ in range(height):
        row = []  # Initialize an empty row for each row
        for _ in range(width):
            row.insert(0, a)  # Insert the Fibonacci number at the beginning of the row
            a, b = b, a + b  # Update Fibonacci sequence variables
        board.append(row)  # Append the row to the board
    board[-1][-1] = BLANK  # Replace the last cell with BLANK
    return board  # Return the Fibonacci sequence table




def generateBackwardRegularTable(width, height):
    # Function to generate a backward regular table
    board = []  # Initialize an empty board
    counter = width * height  # Initialize counter for cell values
    for x in range(width):
        column = []  # Initialize an empty column for each row
        for y in range(height):
            column.append(counter)  # Append the counter value to the column
            counter -= 1  # Decrement counter for the next cell
        board.append(column)  # Append the column to the board
    board[width - 1][height - 1] = BLANK  # Replace the last cell with BLANK
    return board


if __name__ == '__main__':
    main()


pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
